In [16]:
import argparse
import os
import random
import time
import warnings
import sys
import pickle as pk

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.dataset import Dataset 

%run autoencoder.ipynb
%run Train.ipynb

In [14]:
train_data_dir = '../Data/train_dict.pk'
val_data_dir = '../Data/test_dict.pk'

In [12]:
parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
parser.add_argument('--data', default='./data', metavar='DIR', help='path to dataset')
parser.add_argument('--workers', default=1, type=int, metavar='N', help='number of data loading workers (default: 4)')
parser.add_argument('--epochs', default=120, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--epoch_decay', default=30, type=int, metavar='N', help='adjust learning rate per N epochs')
parser.add_argument('--start_epoch', default=0, type=int, metavar='N', help='manual epoch number (useful on restarts)')
parser.add_argument('--batch_size', default=32, type=int, metavar='N', help='mini-batch size (default: 256)')
parser.add_argument('--lr', default=0.001, type=float, metavar='LR', help='initial learning rate')
parser.add_argument('--weight_decay', default=0, type=float, metavar='W', help='weight decay (default: 1e-4)')
parser.add_argument('--print_freq', default=10, type=int, metavar='N', help='print frequency (default: 10)')
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--evaluate', default=False,  dest='evaluate', action='store_true', help='evaluate model')
parser.add_argument('--pretrained', default=False, dest='pretrained', action='store_true', help='use pre-trained model')
parser.add_argument('--seed', default=None, type=int, help='seed for initializing training. ')
parser.add_argument('--gpu', default=True, dest='gpu', action='store_true', help='enable GPU.')
parser.add_argument('--resnet', default=False, dest='resnet', action='store_true', help='train with resnet18')
parser.add_argument('--logging',default= True, help ='logging')
parser.add_argument('--num_classes',default = 2, type=int, help='# of classes')

best_acc1 = 0

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.to_tensor = transforms.ToTensor()
        fo = open(data_dir,'rb')
        self.data_info = pk.load(fo)
        fo.close()
        self.data_arr = self.data_info['data']
        self.label_arr = self.data_info['label']
        
        self.data_len = len(self.data_info['label'])
        
    
    def __getitem__(self, index):
        single_data_name = self.data_arr[index]
        
        data_as_tensor = self.to_tensor(single_data_name)
        single_data_label = self.label_arr[index]
        
        return (data_as_tensor, single_data_label)
    
    def __len__(self):
        return self.data_len
        

In [15]:
def main():
    global args, best_acc1
    args = parser.parse_args()

    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        cudnn.deterministic = True
        warnings.warn('You have chosen to seed training. '
                      'This will turn on the CUDNN deterministic setting, '
                      'which can slow down your training considerably! '
                      'You may see unexpected behavior when restarting '
                      'from checkpoints.')

    # create model
    model = autoencoder()
    if args.gpu:
        print("=> gpu in now working")
        model = model.cuda()
        model = torch.nn.DataParallel(model).cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.Adam(model.parameters(), args.lr, betas=(0.9, 0.999))

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})".format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    my_train_dataset = CustomDataset(train_data_dir)
    my_val_dataset = CustomDataset(val_data_dir)
    
    train_loader = torch.utils.data.DataLoader(dataset = my_train_dataset, batch_size = 128, shuffle =True)
    val_loader = torch.utils.data.DataLoader(dataset= my_val_dataset, batch_size = 128, shuffle= False)

    if args.evaluate:
        validate(val_loader, model, criterion, args.gpu, args.print_freq, f)
        return
    
    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, args.lr, epoch, args.epoch_decay)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args.gpu, args.print_freq)

        # evaluate on validation set
        acc1 = validate(val_loader, model, criterion, args.gpu, args.print_freq)

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_acc1': best_acc1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model = 'autoencoder')


In [17]:
main()

usage: ipykernel_launcher.py [-h] [--data DIR] [--workers N] [--epochs N]
                             [--epoch_decay N] [--start_epoch N]
                             [--batch_size N] [--lr LR] [--weight_decay W]
                             [--print_freq N] [--resume PATH] [--evaluate]
                             [--pretrained] [--seed SEED] [--gpu] [--resnet]
                             [--logging LOGGING] [--num_classes NUM_CLASSES]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/fredrickang/Library/Jupyter/runtime/kernel-9a341abf-7d25-4c8a-9c1d-e5ab548dd1fb.json


SystemExit: 2

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
